### Real Case Study



### Instance generation

In [1]:
GRID_SIZE = 9
FILENAME = 'realCase'

In [4]:
from src.realisticCase import RealisticCaseInstanceGenerator, RealisticCaseSolutionData
from src.witnessproblem import Instance
import random

random.seed(1234)

STATIC_WITNESSES = 25
TESTIMONIES_PER_LYING_WITNESS = 5
BASE_SIMILARITIES = [0, 0.1, 0.2, 0.3, 0.4, 0.5]
LYING_WITNESSES = [0,1,2,3,4,5]

NUMBER_OF_INSTANCES = len(BASE_SIMILARITIES) * len(LYING_WITNESSES)

instances = []
routes: list[RealisticCaseSolutionData] = []
for base_similarity in BASE_SIMILARITIES:
    for lying_witnesses in LYING_WITNESSES:
        instance_generator = RealisticCaseInstanceGenerator(
            base_similarity=base_similarity, 
            grid_size=GRID_SIZE, 
            lying_witnesses=lying_witnesses,
            static_witnesses=STATIC_WITNESSES,
            testimonies_per_lying_witness=TESTIMONIES_PER_LYING_WITNESS)
        instance, actor_route = instance_generator.generateInstance()

        instances.append(instance)
        routes.append(RealisticCaseSolutionData(base_similarity,lying_witnesses,actor_route))

with open(f"instances/{FILENAME}.txt",'w') as instance_file, open(f"solutions/{FILENAME}.txt",'w') as solution_file:
    instance_file.write(Instance.schema().dumps(instances, many=True))
    solution_file.write(RealisticCaseSolutionData.schema().dumps(routes, many=True))

10
13
15
10
16
15
11
14
14
15
9
11
18
12
18
9
13
14
11
12
13
18
18
10
9
9
10
11
18
12
18
15
18
18
13
10


In [1]:
from src.geneticAlgorithm.geneticAlgorithm import GeneticAlgorithm
from src.solutions.greedy import GreedyDistance, GreedyTestimonies, PersistentGreedyAlgorithm

RUNS_PER_INSTANCE = 10
RUN_ONCE = 1

genetic_algorithms = [
    (PersistentGreedyAlgorithm(GreedyDistance()), RUN_ONCE),
    (GreedyTestimonies(), RUNS_PER_INSTANCE),
    (GeneticAlgorithm(
        populationSize=150,
        numGenerations = 100,
        mutationProbability=0.651,
        crossoverRate=0.8321,
        localSearchProb=lambda x,y: 0
        ), RUNS_PER_INSTANCE),
    (GeneticAlgorithm(
        populationSize=150,
        numGenerations = 100,
        mutationProbability=0.7194,
        crossoverRate=0.7344,
        localSearchProb=lambda step,totalSteps: step/totalSteps * 0.2104
        ), RUNS_PER_INSTANCE)]



In [3]:
import src.geneticAlgorithm.algorithmTestUtils as tst
from src.geneticAlgorithm.algorithmDataGatherer import RealisticInstanceDataGatherer

corners = [0, GRID_SIZE-1, GRID_SIZE**2-1, GRID_SIZE**2-GRID_SIZE]

dataGatherer = lambda a,b,c: RealisticInstanceDataGatherer(a,b,c,end_vertices=corners)
 
tst.executeAlgorithms(genetic_algorithms, FILENAME, dataGatherer, startIn=0, endIn=2)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Starting Execution at 20:46
Calculated the initial population
----Finished instance #0 at 20:46
Calculated the initial population
----Finished instance #1 at 20:46


In [3]:
experiments_filename = "realCase"

ROWS = len(BASE_SIMILARITIES)
COLUMNS = len(LYING_WITNESSES)
ALGORITHMS = ['Time-based', 'Testimony-based', 'GA', 'MA']

bold_string, bold_string_end = "$\\bf{", "}$"

with open(f"solutions/{experiments_filename}.csv") as f:
    f.readline()


    for algorithm_name in ALGORITHMS:

        print(algorithm_name)

        line = f.readline().split(',')
        INSTANCES = (len(line) - 1)//5

        algorithm_mean_hits_sum = 0

        x,y = 0,0
        print(f"$s={BASE_SIMILARITIES[x]}$", end='')
        for i in range(1, len(line), 5):
            mean = float(line[i])
            maximum = float(line[i+1])
            mean_hits = float(line[i+2])
            real_fitness = float(line[i+3])
            time = float(line[i+4])

            algorithm_mean_hits_sum += mean_hits

            equal_than_original = real_fitness < mean
            print(f" & {bold_string if equal_than_original else ''}{mean_hits}{bold_string_end if equal_than_original else ''}", end='')

            y += 1
            if y >= COLUMNS:
                y = 0
                print(' \\\\')
                x += 1
                if x < ROWS:
                    print(f"$s={BASE_SIMILARITIES[x]}$", end='')
                
        print(f"MEAN HITS: {algorithm_mean_hits_sum/36}")
        print('------------------------------')


Time-based
$s=0$ & 1.0 & 0.0MEAN HITS: 0.027777777777777776
------------------------------
Testimony-based
$s=0$ & 0.5 & 0.0MEAN HITS: 0.013888888888888888
------------------------------
GA
$s=0$ & 0.8 & 0.4MEAN HITS: 0.03333333333333334
------------------------------
MA
$s=0$ & 0.9 & 0.3MEAN HITS: 0.03333333333333333
------------------------------
